#### En este ejemplo evaluamos tres modelos distintos pre-entrenados y finetuned de BERT

In [29]:
import torch
import logging
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset

# Configurar el logging para evitar mensajes de advertencia
logging.getLogger("transformers").setLevel(logging.ERROR)

# Cargar el conjunto de datos de SST-2
dataset = load_dataset("imdb")

# Mezclar el conjunto de datos
dataset = dataset.shuffle(seed=42)

# Cargar los modelos y los tokenizers
model1_name = "distilbert-base-uncased-finetuned-sst-2-english"
model2_name = "apple/ane-distilbert-base-uncased-finetuned-sst-2-english"
model3_name = "echarlaix/bert-large-uncased-whole-word-masking-finetuned-sst-2"

tokenizer1 = DistilBertTokenizer.from_pretrained(model1_name)
tokenizer2 = DistilBertTokenizer.from_pretrained(model2_name)
tokenizer3 = BertTokenizer.from_pretrained(model3_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model1 = DistilBertForSequenceClassification.from_pretrained(model1_name, ignore_mismatched_sizes=True).to(device)
model2 = DistilBertForSequenceClassification.from_pretrained(model2_name, ignore_mismatched_sizes=True).to(device)
model3 = BertForSequenceClassification.from_pretrained(model3_name, ignore_mismatched_sizes=True).to(device)

# Función para hacer inferencias
def evaluate_model(model, tokenizer, texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1).cpu().numpy()  # Convertir a numpy
    print("model:", model, "predictions", predictions)
    return predictions

# Evaluar los modelos
# Evaluate the model on the test set
texts = dataset['test']['text'][:100]  # Get the first 100 reviews
labels = dataset['test']['label'][:100]  # Corresponding labels

predictions1 = evaluate_model(model1, tokenizer1, texts)
predictions2 = evaluate_model(model2, tokenizer2, texts)
predictions3 = evaluate_model(model3, tokenizer3, texts)

# Comparar resultados
correct_predictions1 = (predictions1 == labels).sum()
correct_predictions2 = (predictions2 == labels).sum()
correct_predictions3 = (predictions3 == labels).sum()

print(f"Modelo 1 (DistilBERT): {correct_predictions1}/100 correctos")
print(f"Modelo 2 (ANE DistilBERT): {correct_predictions2}/100 correctos")
print(f"Modelo 3 (BERT): {correct_predictions3}/100 correctos")

# Función para imprimir ejemplos correctos e incorrectos
def print_examples(predictions, labels, texts, model_name):
    correct_indices = [i for i in range(len(predictions)) if predictions[i] == labels[i]]
    incorrect_indices = [i for i in range(len(predictions)) if predictions[i] != labels[i]]

    print(f"\nEjemplos correctos para {model_name}:")
    for idx in correct_indices[:2]:  # Imprimir 2 ejemplos correctos
        print(f"Texto: {texts[idx]}")
        print(f"Predicción: {predictions[idx]}, Etiqueta: {labels[idx]}\n")

    print(f"Ejemplos incorrectos para {model_name}:")
    for idx in incorrect_indices[:2]:  # Imprimir 2 ejemplos incorrectos
        print(f"Texto: {texts[idx]}")
        print(f"Predicción: {predictions[idx]}, Etiqueta: {labels[idx]}\n")

# Imprimir ejemplos para todos los modelos
print_examples(predictions1, labels, texts, "Modelo 1 (DistilBERT)")
print_examples(predictions2, labels, texts, "Modelo 2 (ANE DistilBERT)")
print_examples(predictions3, labels, texts, "Modelo 3 (BERT)")


model: DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=F

### Inspeccionamos el dataset GLUE

In [ ]:
# Importar las bibliotecas
from datasets import load_dataset

# "MRPC" se refiere a Microsoft Research Paraphrase Corpus, 
# que es uno de los subtareas dentro del conjunto de datos 
# GLUE (General Language Understanding Evaluation). 
# MRPC está diseñado para evaluar la capacidad de los modelos de 
# lenguaje para identificar si dos oraciones son paráfrasis una de la otra.
glue_dataset = load_dataset("glue", "mrpc")

# Ver las características del dataset
print(glue_dataset)

# Mostrar ejemplos
print(glue_dataset['train'][0])  # Mostrar el primer ejemplo del conjunto de entrenamiento


In [ ]:
import torch
from datasets import load_dataset

# SST-2, que significa Stanford Sentiment Treebank 2. 
# Esta tarea se centra en la clasificación de sentimientos 
sst2_dataset = load_dataset("glue", "sst2")

# Ver las características del dataset
print(sst2_dataset)

# Mostrar ejemplos del conjunto de entrenamiento
print("Ejemplo del conjunto de entrenamiento:")
print(sst2_dataset['train'][0])  # Mostrar el primer ejemplo del conjunto de entrenamiento


#### Evaluación de CHATGPT 4O mini en SuperGLUE en la tarea RTE 
##### evalúa la comprensión del lenguaje natural, determinando si una hipótesis se puede inferir de una premisa dada
##### Probar con distintos modelos y con DEBAG = True

In [ ]:
import os
import json
import numpy as np
from datasets import load_dataset
from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import time
import random

def load_rte_data(split="validation", n_samples=20):
    """
    Carga el dataset RTE de SuperGLUE
    """
    dataset = load_dataset(
        "super_glue", 
        "rte",
        trust_remote_code=True
    )[split]
    
    if n_samples:
        dataset = dataset.select(range(min(n_samples, len(dataset))))
    return dataset

def format_prompt(premise, hypothesis):
    """
    Formatea el prompt para la tarea RTE con instrucciones más claras
    """
    return f"""Dado el siguiente par de oraciones, tu tarea es determinar si la segunda oración (hipótesis) 
se puede inferir lógicamente de la primera oración (premisa).

Premisa: {premise}

Hipótesis: {hypothesis}

Si la hipótesis se puede inferir lógicamente de la premisa, responde exactamente con la palabra: entailment
Si la hipótesis NO se puede inferir lógicamente de la premisa, responde exactamente con la palabra: not_entailment

Responde solo con una de estas dos palabras: entailment o not_entailment
Respuesta:"""

def get_model_prediction(client, premise, hypothesis, debug=True):
    """
    Obtiene la predicción del modelo usando la API de OpenAI
    """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k", #"gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Eres un asistente experto en lógica y razonamiento. Tu tarea es determinar si una hipótesis se puede inferir lógicamente de una premisa dada."},
                {"role": "user", "content": format_prompt(premise, hypothesis)}
            ],
            temperature=0,
            max_tokens=10
        )
        prediction = response.choices[0].message.content.strip().lower()
        
        if debug:
            print(f"\nPremisa: {premise}")
            print(f"Hipótesis: {hypothesis}")
            print(f"Predicción raw: {prediction}")
        
        # Normalización más estricta de la respuesta
        if prediction == 'entailment':
            return 'entailment'
        elif prediction == 'not_entailment':
            return 'not_entailment'
        else:
            print(f"⚠️ Respuesta no válida del modelo: {prediction}")
            return None
    
    except Exception as e:
        print(f"Error en la API: {e}")
        time.sleep(20)
        return None

def evaluate_model(client, dataset, debug=True):
    """
    Evalúa el modelo en el dataset RTE con capacidades de depuración
    """
    predictions = []
    true_labels = []
    examples_info = []
    
    # Label map correcto según el dataset de SuperGLUE
    label_map = {0: 'entailment', 1: 'not_entailment'}
    
    for idx, example in enumerate(tqdm(dataset)):
        premise = example['premise']
        hypothesis = example['hypothesis']
        true_label = label_map[example['label']]
        
        pred = get_model_prediction(client, premise, hypothesis, debug=debug)
        
        if pred is not None:
            predictions.append(pred)
            true_labels.append(true_label)
            
            # Guardar información del ejemplo para análisis
            examples_info.append({
                'idx': idx,
                'premise': premise,
                'hypothesis': hypothesis,
                'true_label': true_label,
                'predicted': pred,
                'correct': pred == true_label
            })
            
        time.sleep(1)
    
    return predictions, true_labels, examples_info

def analyze_results(examples_info):
    """
    Analiza los resultados de la evaluación en detalle
    """
    correct_examples = [ex for ex in examples_info if ex['correct']]
    incorrect_examples = [ex for ex in examples_info if not ex['correct']]
    
    print(f"\nTotal de ejemplos evaluados: {len(examples_info)}")
    print(f"Ejemplos correctos: {len(correct_examples)}")
    print(f"Ejemplos incorrectos: {len(incorrect_examples)}")
    
    print("\nEjemplos de predicciones incorrectas:")
    for i, example in enumerate(incorrect_examples[:3]):  # Mostrar los primeros 3 errores
        print(f"\nError {i+1}:")
        print(f"Premisa: {example['premise']}")
        print(f"Hipótesis: {example['hypothesis']}")
        print(f"Etiqueta verdadera: {example['true_label']}")
        print(f"Predicción: {example['predicted']}")
        print("-" * 80)

def main():
    # Inicializar cliente de OpenAI
    client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    
    # Cargar datos
    print("Cargando dataset...")
    dataset = load_rte_data(split="validation", n_samples=20)  # Reducir muestras para depuración
    
    # Evaluar modelo
    print("Evaluando modelo...")
    predictions, true_labels, examples_info = evaluate_model(client, dataset, debug=False)
    
    # Calcular métricas
    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions)
    
    # Analizar resultados
    analyze_results(examples_info)
    
    # Guardar resultados detallados
    results = {
        "accuracy": float(accuracy),
        "classification_report": report,
        "examples_info": examples_info
    }
    
    with open("evaluation_results_detailed.json", "w", encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print("\nResultados finales:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nReporte de clasificación:")
    print(report)

if __name__ == "__main__":
    main()